In [ ]:
threshold = 50
fastp = 0

### ShigaTyper Shigella serotype prediction

In [ ]:
import datetime
now = datetime.datetime.now().strftime("%Y%m%d")

In [ ]:
# Load samples for analysis
# identify sample names for the pair-end reads
import glob
import os
import pandas as pd 
files = glob.glob("*.fastq.gz")
names = []
for file in files:
    names.append(file[:file.find("_")])
samples = list(set(names))
samples.sort()
sizes = []
for Sample in samples:
    size = 0
    for file in files:
        if file.startswith(Sample+"_"):
            size += os.path.getsize(file)
    sizes.append(round(size/1024000, 1))

In [ ]:
# If you need to change the directory/file for the Shigella reference sequence database:
# Remove "#" from the beginning of the 3 lines below starting from "import os" 
# Replace the absolute directory and file name within quotation marks for the reference sequence database 
# in the line beginning with "AbsRef_dir" after "=" 
# For example, '/mnt/hgfs/' is the directory where all shared drives are under VMPlayer, 'Meow' is the name of my
# shared drive,'WGS/references' is the folder directory under the shared drive, and 'ShigellaRef6.fasta' is the
# name of the current reference sequence database. 
# Remove the current line beginning with "ShigellaRef" dictating where the default reference sequence database is.

# import os
# AbsRef_dir = '/mnt/hgfs/Meow/WGS/references/ShigellaRef6.fasta'
# ShigellaRef = os.path.relpath(AbsRef_dir, os.getcwd())

# relative directory of the reference sequence database
ShigellaRef = "../../references/ShigellaRef6.fasta"
if os.path.isfile(ShigellaRef) == False:
    print("Error: reference sequence database does not exist!")
    exit()

# generate a index file for reference
dir_path = os.path.dirname(os.path.realpath(ShigellaRef))
rel_dir = os.path.relpath(dir_path, os.getcwd())
Refname = ShigellaRef[ShigellaRef.rfind('/')+1:ShigellaRef.rfind('.')]
Refname = Refname+".mmi"
mmi_index = os.path.join(rel_dir, Refname)
if os.path.isfile(mmi_index) == False:
    print("building Reference sequence index.......")
    !minimap2 -d $mmi_index $ShigellaRef
# another index needed to generate
fai_index = ShigellaRef+'.fai'
if os.path.isfile(fai_index) == False:
    !samtools faidx $ShigellaRef

In [ ]:
import datetime
now = datetime.datetime.now().strftime("%Y%m%d")
import papermill as pm
outputs = []
for Sample in samples:
    reads = []
    for file in files:
        if file.startswith(Sample+"_"):
            reads.append(file)
    if len(reads) == 2:
        output = Sample + "_" + str(threshold) + "_" + now + ".ipynb"
        outputs.append(output); print(" ")
        pm.execute_notebook('batch_010819.ipynb', output,
                            dict(Sample=Sample, read1 = reads[0], read2 = reads[1], 
                                 ShigellaRef = ShigellaRef, fastp = fastp, threshold = threshold)) 
        !jupyter nbconvert --to html $output
    else: print('\n'+Sample + " does not have pair-end reads! Please re-examine.")

In [ ]:
# if there are fastq files also process them
files = glob.glob("*.fastq")
names = []
for file in files:
    names.append(file[:file.find("_")])
fastqsamples = list(set(names))
fastqsamples.sort()
samples = samples + fastqsamples
for Sample in fastqsamples:
    size = 0
    for file in files:
        if file.startswith(Sample+"_"):
            size += os.path.getsize(file)
    sizes.append(round(size/1024000, 1))

for Sample in fastqsamples:
    reads = []
    for file in files:
        if file.startswith(Sample+"_"):
            reads.append(file)
    if len(reads) == 2:
        output = Sample + "_" + str(threshold) + "_" + now + ".ipynb"
        outputs.append(output); print(" ")
        pm.execute_notebook('batch_010819.ipynb', output,
                            dict(Sample=Sample, read1 = reads[0], read2 = reads[1], 
                                 ShigellaRef = ShigellaRef, fastp = fastp, threshold = threshold)) 
        !jupyter nbconvert --to html $output
    else: print('\n'+Sample + " does not have pair-end reads! Please re-examine.")

In [ ]:
print("Date of analysis:", now)

try: samples
except: samples = []
    
if len(samples) > 0:
    sizetable = pd.DataFrame({'Sample': samples, 'Size (MB)': sizes})

    print("Threshold level for gene coverage: ", threshold, "%")
    print(len(outputs), " samples were analyzed: \n")
    samples = []
    predictions = []
    pINV = []
    ShigaToxin=[]
    Enterotoxin=[]

    for output in outputs:
        nb = pm.read_notebook(output)
        sampleIndex = nb.dataframe.index[nb.dataframe['name']=="Sample"].tolist()[0]
        predictionIndex = nb.dataframe.index[nb.dataframe['name']=="prediction"].tolist()[0]
        pINVindex = nb.dataframe.index[nb.dataframe['name']=="pINV"].tolist()[0]
        stxIndex = nb.dataframe.index[nb.dataframe['name']=="ShigaToxin"].tolist()[0]
        enteroIndex = nb.dataframe.index[nb.dataframe['name']=='Enterotoxin'].tolist()[0]
        samples.append(nb.dataframe.iloc[sampleIndex]['value'])
        predictions.append(nb.dataframe.iloc[predictionIndex]['value'])
        pINV.append(nb.dataframe.iloc[pINVindex]['value'])
        ShigaToxin.append(nb.dataframe.iloc[stxIndex]['value'])
        Enterotoxin.append(nb.dataframe.iloc[enteroIndex]['value'])
        os.remove(output) # delete notebook to keep directory tidy
  
    table = pd.DataFrame({'Sample': samples, 'Serotype prediction': predictions,'Invasion plasmid': pINV,
                          'Shiga Toxin': ShigaToxin, 'Enterotoxin': Enterotoxin})
    final = pd.merge(left=sizetable, right = table, left_on="Sample", right_on = "Sample", how = 'left')
    final = final.sort_values(by=['Sample'])
    from IPython.display import display, HTML
    display(HTML(final.to_html(index=False)))

In [ ]:
# Process assembled genomes in fasta format
files = glob.glob("*.fasta")
names = []
for file in files:
    names.append(file[:file.rfind(".fasta")])
fastasamples = list(set(names))
fastasamples.sort()

fastasizes = []
for Sample in fastasamples:
    size = os.path.getsize(file)
    fastasizes.append(round(size/1024000, 1))

fastaoutputs =[]
for Sample in fastasamples:
    output = Sample + "_" + str(threshold) + "_" + now + ".ipynb"
    fastaoutputs.append(output); print(" ")
    pm.execute_notebook('batch_010819.ipynb', output,
                        dict(Sample=Sample, read1 = Sample+'.fasta', 
                                 ShigellaRef = ShigellaRef, fastp = fastp, threshold = threshold)) 
    !jupyter nbconvert --to html $output

### Summary of serotype prediction results:

In [ ]:
# Display final table for assembled contigs
try: fastasamples
except: fastasamples = []

if len(fastasamples) > 0:
    fastasize = pd.DataFrame({'Sample': fastasamples, 'Size (MB)': fastasizes})

    print("Threshold level for gene coverage: ", threshold, "%")
    print(len(fastaoutputs), " samples were analyzed: \n")
    samples = []
    predictions = []
    pINV = []
    ShigaToxin=[]
    Enterotoxin=[]

    for output in fastaoutputs:
        nb = pm.read_notebook(output)
        sampleIndex = nb.dataframe.index[nb.dataframe['name']=="Sample"].tolist()[0]
        predictionIndex = nb.dataframe.index[nb.dataframe['name']=="prediction"].tolist()[0]
        pINVindex = nb.dataframe.index[nb.dataframe['name']=="pINV"].tolist()[0]
        stxIndex = nb.dataframe.index[nb.dataframe['name']=="ShigaToxin"].tolist()[0]
        enteroIndex = nb.dataframe.index[nb.dataframe['name']=='Enterotoxin'].tolist()[0]
        samples.append(nb.dataframe.iloc[sampleIndex]['value'])
        predictions.append(nb.dataframe.iloc[predictionIndex]['value'])
        pINV.append(nb.dataframe.iloc[pINVindex]['value'])
        ShigaToxin.append(nb.dataframe.iloc[stxIndex]['value'])
        Enterotoxin.append(nb.dataframe.iloc[enteroIndex]['value'])
        os.remove(output) # delete notebook to keep directory tidy
  
    fastatable = pd.DataFrame({'Sample': samples, 'Serotype prediction': predictions,
                               'Invasion plasmid': pINV, 'Shiga Toxin': ShigaToxin, 'Enterotoxin': Enterotoxin})
    fastafinal = pd.merge(left=fastasize, right = fastatable, left_on="Sample", right_on = "Sample", how = 'left')
    fastafinal = fastafinal.sort_values(by=['Sample'])
    from IPython.display import display, HTML
    display(HTML(fastafinal.to_html(index=False)))


#print(nb.dataframe.iloc[3, 1], '\n')

In [ ]:
from IPython.display import HTML
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
The raw code for this notebook is by default hidden for easier reading.
To toggle on/off the raw code, click <a href="javascript:code_toggle()">here</a>.''')